In [1]:
import os
import snowflake.snowpark.functions as F

In [2]:
path_utils = os.path.join(os.getcwd(),'Utils','utils.ipynb')
%run $path_utils

####################################################
Available packages:
    from snowflake.snowpark.session import Session
    from snowflake.snowpark import functions as F
    from snowflake.snowpark.types import *
    from datetime import datetime

    import configparser
    import os

####################################################

Available functions, for details use help(<function_name>:
    get_snowpark_session
    set_schema
    read_csv_from_stage
    create_hz_dim
    get_field_list



In [3]:
session = get_snowpark_session()
set_schema('PZ_CLEAR_STRATEGY')

[Row(status='Statement executed successfully.')]

In [12]:
csv_schema = StructType([
    StructField("id", StringType()),
    StructField("match_event_id", StringType()),
    StructField("location_x", StringType()),
    StructField("location_y", StringType()),
    StructField("remaining_min", StringType()),
    StructField("power_of_shot", StringType()),
    StructField("knockout_match", StringType()),
    StructField("game_season", StringType()),
    StructField("remaining_sec", StringType()),
    StructField("distance_of_shot", StringType()),
    StructField("is_goal", StringType()),
    StructField("area_of_shot", StringType()),
    StructField("shot_basics", StringType()),
    StructField("range_of_shot", StringType()),
    StructField("team_name", StringType()),
    StructField("date_of_game", StringType()),
    StructField("home", StringType()),
    StructField("shot_id_number", StringType()),
    StructField("lat_long", StringType()),
    StructField("type_of_shot", StringType()),
    StructField("type_of_combined_shot", StringType()),
    StructField("match_id", StringType()),
    StructField("team_id", StringType()),
    StructField("remaining_min2", StringType()),
    StructField("power_of_shot3", StringType()),
    StructField("knockout_match4", StringType()),
    StructField("remaining_sec5", StringType()),
    StructField("distance_of_shot6", StringType())]
)

In [22]:
df_csv = read_csv_from_stage(stage_name='RAW_DATA_STAGE',
                             csv_schema=csv_schema,
                             csv_pattern='.*yds_data*[.]csv')

In [23]:
distinct_paths = set([x.PATH for x in df_csv.select('PATH').collect()])

In [24]:
for path in distinct_paths:
    print(path)
    session.sql(f"DELETE FROM football_data WHERE path = '{path}'").show()

raw_data_stage/yds_data.csv
----------------------------
|"number of rows deleted"  |
----------------------------
|30697                     |
----------------------------



In [25]:
df_csv.write.mode("append").saveAsTable('football_data')

## External game data

In [31]:
csv_game_schema = StructType([
    StructField("season_id", StringType()),
    StructField("team_id_home", StringType()),
    StructField("team_abbreviation_home", StringType()),
    StructField("team_name_home", StringType()),
    StructField("game_id", StringType()),
    StructField("game_date", StringType()),
    StructField("matchup_home", StringType()),
    StructField("wl_home", StringType()),
    StructField("min", StringType()),
    StructField("fgm_home", StringType()),
    StructField("fga_home", StringType()),
    StructField("fg_pct_home", StringType()),
    StructField("fg3m_home", StringType()),
    StructField("fg3a_home", StringType()),
    StructField("fg3_pct_home", StringType()),
    StructField("ftm_home", StringType()),
    StructField("fta_home", StringType()),
    StructField("ft_pct_home", StringType()),
    StructField("oreb_home", StringType()),
    StructField("dreb_home", StringType()),
    StructField("reb_home", StringType()),
    StructField("ast_home", StringType()),
    StructField("stl_home", StringType()),
    StructField("blk_home", StringType()),
    StructField("tov_home", StringType()),
    StructField("pf_home", StringType()),
    StructField("pts_home", StringType()),
    StructField("plus_minus_home", StringType()),
    StructField("video_available_home", StringType()),
    StructField("team_id_away", StringType()),
    StructField("team_abbreviation_away", StringType()),
    StructField("team_name_away", StringType()),
    StructField("matchup_away", StringType()),
    StructField("wl_away", StringType()),
    StructField("fgm_away", StringType()),
    StructField("fga_away", StringType()),
    StructField("fg_pct_away", StringType()),
    StructField("fg3m_away", StringType()),
    StructField("fg3a_away", StringType()),
    StructField("fg3_pct_away", StringType()),
    StructField("ftm_away", StringType()),
    StructField("fta_away", StringType()),
    StructField("ft_pct_away", StringType()),
    StructField("oreb_away", StringType()),
    StructField("dreb_away", StringType()),
    StructField("reb_away", StringType()),
    StructField("ast_away", StringType()),
    StructField("stl_away", StringType()),
    StructField("blk_away", StringType()),
    StructField("tov_away", StringType()),
    StructField("pf_away", StringType()),
    StructField("pts_away", StringType()),
    StructField("plus_minus_away", StringType()),
    StructField("video_available_away", StringType())]
)

In [33]:
df_game = read_csv_from_stage(stage_name='RAW_DATA_STAGE',
                             csv_schema=csv_game_schema,
                             csv_pattern='.*game*[.]csv')

df_game.write.mode("overwrite").saveAsTable('nba_game_data')